In [1]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle 

In [16]:
with open('yoga_pose_detection.pkl', 'rb') as f:
    model = pickle.load(f)

In [3]:
mp_pose = mp.solutions.pose # Mediapipe Solutions
mp_drawing = mp.solutions.drawing_utils # Drawing helpers

In [4]:
def inFrame(lst):
	if (lst[28].visibility > 0.6 or lst[27].visibility > 0.6) and (lst[15].visibility>0.6 or lst[16].visibility>0.6):
		return True 
	return False

In [20]:
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
# Initiate holistic model
with mp_pose.Pose(min_detection_confidence=0.4, min_tracking_confidence=0.6) as pose:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = pose.process(image)

        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            circle_coord = (350,40)

            if results.pose_landmarks and inFrame(results.pose_landmarks.landmark):


                # Extract Pose landmarks
                poses = results.pose_landmarks.landmark
                pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in poses]).flatten())
                
                # Concate rows
                row = pose_row
                
                # Make Detections
                X = pd.DataFrame([row])
                body_language_class = model.predict(X)[0]
                body_language_prob = model.predict_proba(X)[0]
                #print(model.evaluate(X))
                #print(body_language_prob)
                
            
                # Get status box
                cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
                
                
                # Display Class
                
                cv2.putText(image, 'CLASS'
                            , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                
                if body_language_prob[np.argmax(body_language_prob)] > 0.85:
                    cv2.putText(image, body_language_class.split(' ')[0]
                                , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                    cv2.circle(image, circle_coord, 40, (0, 255, 0), -1)
                else:
                    cv2.putText(image, "no pose"
                                , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                    cv2.circle(image, circle_coord, 40, (0, 0, 255), -1)
                
                # Display Probability
                cv2.putText(image, 'PROB'
                            , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],3))
                            , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                
            
            else: 
                cv2.putText(image, "Make your Full ", (50,35), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),3)
                cv2.putText(image, "body visible", (50,65), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),3)
                cv2.circle(image, circle_coord, 40, (0, 0, 255), -1)
            
        except:
            pass

                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

c:\Users\juani\anaconda3\envs\py310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\juani\anaconda3\envs\py310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\juani\anaconda3\envs\py310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\juani\anaconda3\envs\py310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\juani\anaconda3\envs\py310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\juani\anaconda3\envs\py310\lib\site-

In [19]:
cap.release()
cv2.destroyAllWindows()